In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import keras
from keras.models import load_model
from keras import backend as K
from keras import models
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from keras_tuner.tuners import RandomSearch
import keras_tuner

2022-11-10 15:21:01.817468: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 15:21:01.817558: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 15:21:01.939612: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 15:21:03.745465: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-10 15:21:03.745773: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
from dataset import load_dataset, load_ica_dataset, load_labels, convert_to_epochs
from features import time_series_features, nonlinear_features, entropy_features, hjorth_features, freq_band_features

In [3]:
num_classes = 2
channels = 32
sfreq = 128

In [4]:
#Filtered from SAM40
dataset_ = load_dataset(raw = False)
dataset = convert_to_epochs(dataset_, channels, sfreq)

#ICA filtered
dataset_ica_ = load_ica_dataset(round=1)
dataset_ica = convert_to_epochs(dataset_ica_, channels, sfreq)

#ICA filtered two times
dataset_ica_2_ = load_ica_dataset(round=2)
dataset_ica_2 = convert_to_epochs(dataset_ica_2_, channels, sfreq)

In [7]:
features = time_series_features(dataset, channels)
features_ica = time_series_features(dataset_ica, channels)
features_ica_2 = time_series_features(dataset_ica_2, channels)
# freq_bands = np.array([1, 4, 8, 13, 31, 50])
# features = freq_band_features(dataset, channels, sfreq, freq_bands)
# features = nonlinear_features(dataset, channels)
# features = hjorth_features(dataset, channels, sfreq)
# features = entropy_features(dataset, channels, sfreq)
data = features
data_ica = features_ica
data_ica_2 = features_ica_2

(120, 25, 32, 128)
(120, 25, 32, 128)
(120, 25, 32, 128)


In [5]:
labels = load_labels()
label = pd.concat([labels['t1_math'], labels['t2_math'],
                  labels['t3_math']]).to_numpy()
label = label.repeat(dataset.shape[1])

# Linear Regression

In [7]:
def LinearRegression(data, label):
    K.clear_session()
    x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    lr_clf = LogisticRegression(max_iter=1000).fit(x_train, y_train)
    y_pred = lr_clf.predict(x_test)
    y_true = y_test

    scores_lr = lr_clf.score(x_test, y_test)
    precision_lr = metrics.precision_score(y_true, y_pred, average='macro')
    recall_lr = metrics.recall_score(y_true, y_pred, average='micro')
    f1_score_lr = metrics.f1_score(y_true, y_pred, average='weighted')
    print('accuracy is:', scores_lr)
    print('precision is:', precision_lr)
    print('recall is:', recall_lr)
    print('f1_score is:', f1_score_lr)
    metrics.confusion_matrix(y_true, y_pred)

# KNN Classifier

In [8]:
def KNN(data, label):
    K.clear_session()
    x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.33, random_state=42)
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    neigh_clf = KNeighborsClassifier(n_neighbors=2)
    neigh_clf.fit(x_train, y_train)
    y_pred = neigh_clf.predict(x_test)
    y_true = y_test

    scores_neigh = neigh_clf.score(x_test, y_test)
    precision_neigh = metrics.precision_score(y_true, y_pred, average='macro')
    recall_neigh = metrics.recall_score(y_true, y_pred, average='micro')
    f1_score_neigh = metrics.f1_score(y_true, y_pred, average='weighted')
    print('accuracy is:', scores_neigh)
    print('precision is:', precision_neigh)
    print('recall is:', recall_neigh)
    print('f1_score is:', f1_score_neigh)
    metrics.confusion_matrix(y_true, y_pred)

# SVM Classifier

In [9]:
def SVM(data, label):
    x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
    svm_clf = SVC(C=10, kernel='rbf')
    svm_clf.fit(x_train, y_train)
    y_pred = svm_clf.predict(x_test)
    y_pred_train = svm_clf.predict(x_train)
    y_true = y_test

    scores_svm_train = svm_clf.score(x_train, y_train)
    precision_svm_train = metrics.precision_score(y_train, y_pred_train, average='macro')
    recall_svm_train = metrics.recall_score(y_train, y_pred_train, average='micro')
    f1_score_svm_train = metrics.f1_score(y_train, y_pred_train, average='weighted')
    print('accuracy is:', scores_svm_train)
    print('precision is:', precision_svm_train)
    print('recall is:', precision_svm_train)
    print('f1_score is:', f1_score_svm_train)
    metrics.confusion_matrix(y_true, y_pred)


In [10]:
print('Filtered data')
LinearRegression(data,label)
print('\nICA filtered data')
LinearRegression(data_ica,label)
print('\nICA filtered data round 2')
LinearRegression(data_ica_2,label)

Filtered data
accuracy is: 0.6505050505050505
precision is: 0.6490383236134278
recall is: 0.6505050505050505
f1_score is: 0.6505050505050505

ICA filtered data
accuracy is: 0.6474747474747474
precision is: 0.6458736907771612
recall is: 0.6474747474747474
f1_score is: 0.647350051125969

ICA filtered data round 2
accuracy is: 0.6424242424242425
precision is: 0.6424242424242425
recall is: 0.6424242424242425
f1_score is: 0.6427982252159334


In [11]:
print('Filtered data')
KNN(data,label)
print('\nICA filtered data')
KNN(data_ica,label)
print('\nICA filtered data round 2')
KNN(data_ica_2,label)

Filtered data
accuracy is: 0.6161616161616161
precision is: 0.634022736611423
recall is: 0.6161616161616161
f1_score is: 0.5899596648717212

ICA filtered data
accuracy is: 0.5707070707070707
precision is: 0.5724594594594594
recall is: 0.5707070707070707
f1_score is: 0.5426690751092353

ICA filtered data round 2
accuracy is: 0.6212121212121212
precision is: 0.6411022722278381
recall is: 0.6212121212121212
f1_score is: 0.5951278440603571


In [12]:
print('Filtered data')
SVM(data,label)
print('\nICA filtered data')
SVM(data_ica,label)
print('\nICA filtered data round 2')
SVM(data_ica_2,label)

Filtered data
accuracy is: 0.8009950248756219
precision is: 0.8017411274654813
recall is: 0.8017411274654813
f1_score is: 0.8006208232559378

ICA filtered data
accuracy is: 0.7442786069651741
precision is: 0.7483465987173614
recall is: 0.7483465987173614
f1_score is: 0.7423439799870053

ICA filtered data round 2
accuracy is: 0.8402985074626865
precision is: 0.8436974611937724
recall is: 0.8436974611937724
f1_score is: 0.839635206162414


# Neural Network

In [30]:
K.clear_session()
y_v = label
y_v = to_categorical(y_v, num_classes)
x_train, x_test, y_train, y_test = train_test_split(data, y_v, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)

def model_builder(hp):
    model = keras.models.Sequential()
    model.add(keras.Input(shape=(x_train.shape[1],)))

    for i in range(hp.Int('layers', 2, 6)):
      model.add(keras.layers.Dense(units=hp.Int('units_' + str(i), 32, 1024, step=32),
                                    activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])))

    model.add(keras.layers.Dense(num_classes, activation='softmax', name='out'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.adam_v2.Adam(learning_rate=hp_learning_rate),
                loss = "binary_crossentropy",
                metrics=['accuracy'])
    return model


tuner = RandomSearch(
    model_builder,
    objective = 'val_accuracy',
    max_trials = 5,
    executions_per_trial = 2,
    overwrite=True
)

tuner.search_space_summary()

tuner.search(x_train, y_train, epochs = 50, validation_data= [x_val, y_val])

model = tuner.get_best_models(num_models=1)[0]

y_pred = model.predict(x_test)
y_true = y_test
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

scores_dnn = model.evaluate(x_test, y_test, verbose=0)
precision_dnn = metrics.precision_score(y_true, y_pred, average='macro')
recall_dnn = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_dnn = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_dnn[1])
print('precision is:', precision_dnn)
print('recall is:', recall_dnn)
print('f1_score is:', f1_score_dnn)
metrics.confusion_matrix(y_true, y_pred)

Trial 5 Complete [00h 02m 41s]
val_accuracy: 0.565833330154419

Best val_accuracy So Far: 0.8816666603088379
Total elapsed time: 00h 09m 16s
INFO:tensorflow:Oracle triggered exit
19/19 [==============================] - 0s 2ms/step
accuracy is: 0.824999988079071
precision is: 0.8261298421807748
recall is: 0.825
f1_score is: 0.8246763408735239


array([[267,  44],
       [ 61, 228]])

In [31]:
K.clear_session()
y_v = label
y_v = to_categorical(y_v, num_classes)
x_train, x_test, y_train, y_test = train_test_split(data_ica, y_v, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)

def model_builder(hp):
    model = keras.models.Sequential()
    model.add(keras.Input(shape=(x_train.shape[1],)))

    for i in range(hp.Int('layers', 2, 6)):
      model.add(keras.layers.Dense(units=hp.Int('units_' + str(i), 32, 1024, step=32),
                                    activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])))

    model.add(keras.layers.Dense(num_classes, activation='softmax', name='out'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.adam_v2.Adam(learning_rate=hp_learning_rate),
                loss = "binary_crossentropy",
                metrics=['accuracy'])
    return model


tuner = RandomSearch(
    model_builder,
    objective = 'val_accuracy',
    max_trials = 5,
    executions_per_trial = 2,
    overwrite=True
)

tuner.search_space_summary()

tuner.search(x_train, y_train, epochs = 50, validation_data= [x_val, y_val])

model = tuner.get_best_models(num_models=1)[0]

y_pred = model.predict(x_test)
y_true = y_test
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

scores_dnn = model.evaluate(x_test, y_test, verbose=0)
precision_dnn = metrics.precision_score(y_true, y_pred, average='macro')
recall_dnn = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_dnn = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_dnn[1])
print('precision is:', precision_dnn)
print('recall is:', recall_dnn)
print('f1_score is:', f1_score_dnn)
metrics.confusion_matrix(y_true, y_pred)
    

Trial 5 Complete [00h 02m 57s]
val_accuracy: 0.7450000047683716

Best val_accuracy So Far: 0.8266666531562805
Total elapsed time: 00h 10m 44s
INFO:tensorflow:Oracle triggered exit
19/19 [==============================] - 0s 8ms/step
accuracy is: 0.846666693687439
precision is: 0.846427499416349
recall is: 0.8466666666666667
f1_score is: 0.8466973609581956


array([[263,  48],
       [ 44, 245]])

In [8]:
K.clear_session()
y_v = label
y_v = to_categorical(y_v, num_classes)
x_train, x_test, y_train, y_test = train_test_split(data_ica_2, y_v, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)

def model_builder(hp):
    model = keras.models.Sequential()
    model.add(keras.Input(shape=(x_train.shape[1],)))

    for i in range(hp.Int('layers', 2, 6)):
      model.add(keras.layers.Dense(units=hp.Int('units_' + str(i), 32, 1024, step=32),
                                    activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])))

    model.add(keras.layers.Dense(num_classes, activation='softmax', name='out'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.adam_v2.Adam(learning_rate=hp_learning_rate),
                loss = "binary_crossentropy",
                metrics=['accuracy'])
    return model


tuner = RandomSearch(
    model_builder,
    objective = 'val_accuracy',
    max_trials = 5,
    executions_per_trial = 2,
    overwrite=True
)

tuner.search_space_summary()

tuner.search(x_train, y_train, epochs = 50, validation_data= [x_val, y_val])

model = tuner.get_best_models(num_models=1)[0]

y_pred = model.predict(x_test)
y_true = y_test
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

scores_dnn = model.evaluate(x_test, y_test, verbose=0)
precision_dnn = metrics.precision_score(y_true, y_pred, average='macro')
recall_dnn = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_dnn = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_dnn[1])
print('precision is:', precision_dnn)
print('recall is:', recall_dnn)
print('f1_score is:', f1_score_dnn)
metrics.confusion_matrix(y_true, y_pred)

Trial 5 Complete [00h 01m 56s]
val_accuracy: 0.8974999785423279

Best val_accuracy So Far: 0.8974999785423279
Total elapsed time: 00h 09m 32s
INFO:tensorflow:Oracle triggered exit
19/19 [==============================] - 0s 5ms/step
accuracy is: 0.8949999809265137
precision is: 0.8957839939228741
recall is: 0.895
f1_score is: 0.894893803756363


array([[285,  26],
       [ 37, 252]])